In [38]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-17 18:44:53.426181: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 18:44:53.426488: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 18:44:53.426631: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 18:44:53.426815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 18:44:53.426952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [39]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [3]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                        

2023-05-17 15:03:35.290749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 15:03:35.290943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 15:03:35.291066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 15:03:35.291349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 15:03:35.291479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [4]:
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=100, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Epoch 1/100


2023-05-17 15:03:41.303600: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-05-17 15:03:42.753793: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


782/782 [==============================] - 7s 6ms/step - loss: 2.2705 - accuracy: 0.1367 - val_loss: 2.1441 - val_accuracy: 0.2177
Epoch 2/100
782/782 [==============================] - 4s 5ms/step - loss: 2.0634 - accuracy: 0.2140 - val_loss: 1.9059 - val_accuracy: 0.2737
Epoch 3/100
782/782 [==============================] - 4s 5ms/step - loss: 1.8974 - accuracy: 0.2657 - val_loss: 1.8181 - val_accuracy: 0.3277
Epoch 4/100
782/782 [==============================] - 4s 5ms/step - loss: 1.7788 - accuracy: 0.3157 - val_loss: 1.6390 - val_accuracy: 0.3989
Epoch 5/100
782/782 [==============================] - 4s 5ms/step - loss: 1.6987 - accuracy: 0.3476 - val_loss: 1.5518 - val_accuracy: 0.4307
Epoch 6/100
782/782 [==============================] - 4s 5ms/step - loss: 1.6314 - accuracy: 0.3803 - val_loss: 1.5057 - val_accuracy: 0.4511
Epoch 7/100
782/782 [==============================] - 4s 5ms/step - loss: 1.5726 - accuracy: 0.4080 - val_loss: 1.4266 - val_accuracy: 0.4867
Epoch 8/100

782/782 [==============================] - 4s 5ms/step - loss: 0.7238 - accuracy: 0.7506 - val_loss: 0.6713 - val_accuracy: 0.7620
Epoch 59/100
782/782 [==============================] - 4s 6ms/step - loss: 0.7180 - accuracy: 0.7511 - val_loss: 0.6487 - val_accuracy: 0.7712
Epoch 60/100
782/782 [==============================] - 4s 5ms/step - loss: 0.7127 - accuracy: 0.7531 - val_loss: 0.6203 - val_accuracy: 0.7847
Epoch 61/100
782/782 [==============================] - 4s 5ms/step - loss: 0.7064 - accuracy: 0.7539 - val_loss: 0.6241 - val_accuracy: 0.7815
Epoch 62/100
782/782 [==============================] - 4s 5ms/step - loss: 0.7012 - accuracy: 0.7556 - val_loss: 0.6045 - val_accuracy: 0.7868
Epoch 63/100
782/782 [==============================] - 4s 6ms/step - loss: 0.6989 - accuracy: 0.7585 - val_loss: 0.6357 - val_accuracy: 0.7783
Epoch 64/100
782/782 [==============================] - 4s 6ms/step - loss: 0.6904 - accuracy: 0.7622 - val_loss: 0.6102 - val_accuracy: 0.7861
Epoch

In [5]:

# load the pre-saved model
model_10 = load_model('model_10_gpu_Base-CubeAI.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.5236 - accuracy: 0.8181
Test accuracy: 0.8180999755859375


In [6]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI-PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmp2ge0xf38/assets


INFO:tensorflow:Assets written to: /tmp/tmp2ge0xf38/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 15:19:18.816609: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 15:19:18.816632: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 15:19:18.817175: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2ge0xf38
2023-05-17 15:19:18.819507: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 15:19:18.819524: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp2ge0xf38
2023-05-17 15:19:18.826415: I tensorflow/

== Input details ==
name: serving_default_conv2d_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of quantized to int8 model is 82.05%


In [7]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=100,validation_data=(x_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_2 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_12 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_conv2d_13 (QuantizeWr  (None, 32, 32, 32)       9315      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_6 (Quan  (None, 16, 16, 32)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_dropout_8 (QuantizeWr  (None, 16, 16, 32)      

782/782 [==============================] - 12s 15ms/step - loss: 0.4621 - accuracy: 0.8442 - val_loss: 0.5744 - val_accuracy: 0.8198
Epoch 88/100
782/782 [==============================] - 12s 15ms/step - loss: 0.4612 - accuracy: 0.8432 - val_loss: 0.5530 - val_accuracy: 0.8277
Epoch 89/100
782/782 [==============================] - 12s 15ms/step - loss: 0.4615 - accuracy: 0.8441 - val_loss: 0.5732 - val_accuracy: 0.8265
Epoch 90/100
782/782 [==============================] - 12s 15ms/step - loss: 0.4570 - accuracy: 0.8440 - val_loss: 0.5922 - val_accuracy: 0.8171
Epoch 91/100
782/782 [==============================] - 12s 15ms/step - loss: 0.4623 - accuracy: 0.8432 - val_loss: 0.5605 - val_accuracy: 0.8234
Epoch 92/100
782/782 [==============================] - 12s 15ms/step - loss: 0.4658 - accuracy: 0.8434 - val_loss: 0.5790 - val_accuracy: 0.8238
Epoch 93/100
782/782 [==============================] - 12s 15ms/step - loss: 0.4590 - accuracy: 0.8436 - val_loss: 0.6217 - val_accuracy

In [8]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_qat_int8.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Quantization aware training loss:  0.6202415823936462
Quantization aware training accuracy:  0.8119000196456909


INFO:tensorflow:Assets written to: /tmp/tmpj0ynl284/assets


INFO:tensorflow:Assets written to: /tmp/tmpj0ynl284/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 15:39:50.384161: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 15:39:50.384195: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 15:39:50.384343: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpj0ynl284
2023-05-17 15:39:50.389846: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 15:39:50.389863: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpj0ynl284
2023-05-17 15:39:50.412612: I tensorflow/

== Input details ==
name: serving_default_conv2d_12_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of quantized to int8 model is 81.17999999999999%


In [9]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [10]:
c_model_name = 'cifar10_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [11]:

c_model_name = 'cifar10_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

In [14]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_reduced.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=100, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 dropout_20 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_29 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_30 (Conv2D)          (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                 

Epoch 40/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9831 - accuracy: 0.6514 - val_loss: 0.8508 - val_accuracy: 0.6974
Epoch 41/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9706 - accuracy: 0.6586 - val_loss: 0.8727 - val_accuracy: 0.6862
Epoch 42/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9620 - accuracy: 0.6616 - val_loss: 0.8951 - val_accuracy: 0.6815
Epoch 43/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9530 - accuracy: 0.6637 - val_loss: 0.8391 - val_accuracy: 0.7012
Epoch 44/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9463 - accuracy: 0.6676 - val_loss: 0.7983 - val_accuracy: 0.7199
Epoch 45/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9361 - accuracy: 0.6692 - val_loss: 0.8355 - val_accuracy: 0.7019
Epoch 46/100
782/782 [==============================] - 4s 5ms/step - loss: 0.9313 - accuracy: 0.6715 - val_loss: 0.8037 - val_accuracy:

Epoch 97/100
782/782 [==============================] - 4s 5ms/step - loss: 0.6744 - accuracy: 0.7678 - val_loss: 0.5881 - val_accuracy: 0.7992
Epoch 98/100
782/782 [==============================] - 4s 5ms/step - loss: 0.6699 - accuracy: 0.7691 - val_loss: 0.6171 - val_accuracy: 0.7860
Epoch 99/100
782/782 [==============================] - 4s 5ms/step - loss: 0.6609 - accuracy: 0.7699 - val_loss: 0.6067 - val_accuracy: 0.7961
Epoch 100/100
782/782 [==============================] - 4s 5ms/step - loss: 0.6599 - accuracy: 0.7719 - val_loss: 0.6264 - val_accuracy: 0.7876


In [15]:
# load the pre-saved model
model_10 = load_model('model_10_gpu_Base-CubeAI_reduced.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.6264 - accuracy: 0.7876
Test accuracy: 0.7875999808311462


In [16]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI_reduced-PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of reduced base quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmpujqp3jr7/assets


INFO:tensorflow:Assets written to: /tmp/tmpujqp3jr7/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 16:26:17.724843: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 16:26:17.724866: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 16:26:17.725006: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpujqp3jr7
2023-05-17 16:26:17.727111: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 16:26:17.727125: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpujqp3jr7
2023-05-17 16:26:17.732583: I tensorflow/

== Input details ==
name: serving_default_conv2d_28_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of reduced base quantized to int8 model is 78.79%


In [17]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_reduced_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=100,validation_data=(x_test, y_test),
                     callbacks=[checkpoint_callback])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_3 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_33 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_18 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_24 (QuantizeW  (None, 16, 16, 32)       1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_34 (QuantizeWr  (None, 16, 16, 64)      

782/782 [==============================] - 11s 14ms/step - loss: 0.7059 - accuracy: 0.7554 - val_loss: 0.6528 - val_accuracy: 0.7716
Epoch 89/100
782/782 [==============================] - 11s 14ms/step - loss: 0.7052 - accuracy: 0.7544 - val_loss: 0.6227 - val_accuracy: 0.7843
Epoch 90/100
782/782 [==============================] - 11s 14ms/step - loss: 0.6977 - accuracy: 0.7593 - val_loss: 0.6353 - val_accuracy: 0.7807
Epoch 91/100
782/782 [==============================] - 11s 14ms/step - loss: 0.6954 - accuracy: 0.7595 - val_loss: 0.6187 - val_accuracy: 0.7841
Epoch 92/100
782/782 [==============================] - 11s 14ms/step - loss: 0.6941 - accuracy: 0.7611 - val_loss: 0.6153 - val_accuracy: 0.7853
Epoch 93/100
782/782 [==============================] - 11s 14ms/step - loss: 0.6918 - accuracy: 0.7613 - val_loss: 0.6347 - val_accuracy: 0.7773
Epoch 94/100
782/782 [==============================] - 11s 15ms/step - loss: 0.6896 - accuracy: 0.7618 - val_loss: 0.6413 - val_accuracy

In [25]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_reduced_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_reduced_qat_int8.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Quantization aware training loss:  0.6596236228942871
Quantization aware training accuracy:  0.7692000269889832


INFO:tensorflow:Assets written to: /tmp/tmpmeh1heq3/assets


INFO:tensorflow:Assets written to: /tmp/tmpmeh1heq3/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 17:13:42.017819: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 17:13:42.017853: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 17:13:42.017993: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpmeh1heq3
2023-05-17 17:13:42.022470: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 17:13:42.022489: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpmeh1heq3
2023-05-17 17:13:42.035169: I tensorflow/

== Input details ==
name: serving_default_conv2d_42_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of quantized to int8 model is 76.85%


In [19]:
c_model_name = 'cifar10_base_reduced_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [20]:
c_model_name = 'cifar10_reduced_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

In [21]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_4Conv.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=100, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 dropout_28 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_39 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 dropout_29 (Dropout)        (None, 8, 8, 64)         

/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
782/782 [==============================] - 4s 5ms/step - loss: 2.3276 - accuracy: 0.1224 - val_loss: 2.1935 - val_accuracy: 0.1866
Epoch 2/100
782/782 [==============================] - 3s 4ms/step - loss: 2.1356 - accuracy: 0.1936 - val_loss: 2.0472 - val_accuracy: 0.2585
Epoch 3/100
782/782 [==============================] - 3s 4ms/step - loss: 2.0183 - accuracy: 0.2382 - val_loss: 1.9070 - val_accuracy: 0.2970
Epoch 4/100
782/782 [==============================] - 3s 4ms/step - loss: 1.8897 - accuracy: 0.2807 - val_loss: 1.7688 - val_accuracy: 0.3588
Epoch 5/100
782/782 [==============================] - 3s 4ms/step - loss: 1.8004 - accuracy: 0.3170 - val_loss: 1.7162 - val_accuracy: 0.3806
Epoch 6/100
782/782 [==============================] - 3s 4ms/step - loss: 1.7315 - accuracy: 0.3484 - val_loss: 1.6685 - val_accuracy: 0.3935
Epoch 7/100
782/782 [==============================] - 3s 4ms/step - loss: 1.6803 - accuracy: 0.3687 - val_loss: 1.5466 - val_accuracy: 0.4393

Epoch 58/100
782/782 [==============================] - 3s 4ms/step - loss: 0.9336 - accuracy: 0.6728 - val_loss: 0.8105 - val_accuracy: 0.7146
Epoch 59/100
782/782 [==============================] - 3s 4ms/step - loss: 0.9252 - accuracy: 0.6785 - val_loss: 0.8076 - val_accuracy: 0.7160
Epoch 60/100
782/782 [==============================] - 3s 4ms/step - loss: 0.9157 - accuracy: 0.6781 - val_loss: 0.9021 - val_accuracy: 0.6864
Epoch 61/100
782/782 [==============================] - 3s 4ms/step - loss: 0.9127 - accuracy: 0.6816 - val_loss: 0.9223 - val_accuracy: 0.6768
Epoch 62/100
782/782 [==============================] - 3s 4ms/step - loss: 0.9069 - accuracy: 0.6833 - val_loss: 0.7912 - val_accuracy: 0.7223
Epoch 63/100
782/782 [==============================] - 3s 4ms/step - loss: 0.8983 - accuracy: 0.6863 - val_loss: 0.7956 - val_accuracy: 0.7188
Epoch 64/100
782/782 [==============================] - 3s 4ms/step - loss: 0.8950 - accuracy: 0.6858 - val_loss: 0.7905 - val_accuracy:

In [22]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI_4Conv-PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of reduced base quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmpuefmuyzg/assets


INFO:tensorflow:Assets written to: /tmp/tmpuefmuyzg/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 16:55:53.976100: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 16:55:53.976127: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 16:55:53.976269: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpuefmuyzg
2023-05-17 16:55:53.978266: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 16:55:53.978284: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpuefmuyzg
2023-05-17 16:55:53.983492: I tensorflow/

== Input details ==
name: serving_default_conv2d_38_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of reduced base quantized to int8 model is 77.4%


In [23]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_4Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=100,validation_data=(x_test, y_test),
                     callbacks=[checkpoint_callback])

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_4 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_42 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_24 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_32 (QuantizeW  (None, 16, 16, 32)       1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_43 (QuantizeWr  (None, 16, 16, 64)      

Epoch 34/100
782/782 [==============================] - 10s 12ms/step - loss: 1.1498 - accuracy: 0.5899 - val_loss: 1.0357 - val_accuracy: 0.6377
Epoch 35/100
782/782 [==============================] - 10s 12ms/step - loss: 1.1415 - accuracy: 0.5932 - val_loss: 1.0471 - val_accuracy: 0.6338
Epoch 36/100
782/782 [==============================] - 10s 12ms/step - loss: 1.1304 - accuracy: 0.5967 - val_loss: 1.0293 - val_accuracy: 0.6400
Epoch 37/100
782/782 [==============================] - 10s 12ms/step - loss: 1.1189 - accuracy: 0.6012 - val_loss: 1.0865 - val_accuracy: 0.6098
Epoch 38/100
782/782 [==============================] - 10s 12ms/step - loss: 1.1104 - accuracy: 0.6040 - val_loss: 0.9947 - val_accuracy: 0.6459
Epoch 39/100
782/782 [==============================] - 10s 13ms/step - loss: 1.0967 - accuracy: 0.6117 - val_loss: 0.9634 - val_accuracy: 0.6558
Epoch 40/100
782/782 [==============================] - 10s 13ms/step - loss: 1.0884 - accuracy: 0.6124 - val_loss: 0.9859 -

In [24]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_4Conv_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_4Conv_qat_int8.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Quantization aware training loss:  0.6596236228942871
Quantization aware training accuracy:  0.7692000269889832


INFO:tensorflow:Assets written to: /tmp/tmp95_4bf4e/assets


INFO:tensorflow:Assets written to: /tmp/tmp95_4bf4e/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 17:13:29.184972: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 17:13:29.185005: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 17:13:29.185144: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp95_4bf4e
2023-05-17 17:13:29.190065: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 17:13:29.190082: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp95_4bf4e
2023-05-17 17:13:29.205439: I tensorflow/

== Input details ==
name: serving_default_conv2d_42_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of quantized to int8 model is 76.85%


In [26]:
c_model_name = 'cifar10_base_4conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [27]:
c_model_name = 'cifar10_base_4conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

In [31]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_3Conv.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=400, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_49 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 dropout_40 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_50 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 dropout_41 (Dropout)        (None, 8, 8, 64)        

/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/400
782/782 [==============================] - 4s 4ms/step - loss: 2.3278 - accuracy: 0.1217 - val_loss: 2.2279 - val_accuracy: 0.1901
Epoch 2/400
782/782 [==============================] - 3s 4ms/step - loss: 2.1597 - accuracy: 0.1832 - val_loss: 2.0796 - val_accuracy: 0.2470
Epoch 3/400
782/782 [==============================] - 3s 4ms/step - loss: 2.0512 - accuracy: 0.2246 - val_loss: 1.9502 - val_accuracy: 0.3100
Epoch 4/400
782/782 [==============================] - 3s 4ms/step - loss: 1.9420 - accuracy: 0.2638 - val_loss: 1.8353 - val_accuracy: 0.3580
Epoch 5/400
782/782 [==============================] - 3s 4ms/step - loss: 1.8684 - accuracy: 0.2937 - val_loss: 1.7781 - val_accuracy: 0.3683
Epoch 6/400
782/782 [==============================] - 3s 4ms/step - loss: 1.8183 - accuracy: 0.3095 - val_loss: 1.7377 - val_accuracy: 0.3816
Epoch 7/400
782/782 [==============================] - 3s 4ms/step - loss: 1.7751 - accuracy: 0.3276 - val_loss: 1.7001 - val_accuracy: 0.3939

Epoch 58/400
782/782 [==============================] - 3s 4ms/step - loss: 1.1217 - accuracy: 0.6020 - val_loss: 0.9741 - val_accuracy: 0.6631
Epoch 59/400
782/782 [==============================] - 3s 4ms/step - loss: 1.1188 - accuracy: 0.5986 - val_loss: 0.9636 - val_accuracy: 0.6697
Epoch 60/400
782/782 [==============================] - 3s 4ms/step - loss: 1.1136 - accuracy: 0.6054 - val_loss: 0.9680 - val_accuracy: 0.6698
Epoch 61/400
782/782 [==============================] - 3s 4ms/step - loss: 1.1115 - accuracy: 0.6007 - val_loss: 0.9728 - val_accuracy: 0.6630
Epoch 62/400
782/782 [==============================] - 3s 4ms/step - loss: 1.0993 - accuracy: 0.6089 - val_loss: 0.9634 - val_accuracy: 0.6633
Epoch 63/400
782/782 [==============================] - 3s 4ms/step - loss: 1.0936 - accuracy: 0.6099 - val_loss: 0.9906 - val_accuracy: 0.6493
Epoch 64/400
782/782 [==============================] - 3s 4ms/step - loss: 1.0884 - accuracy: 0.6156 - val_loss: 0.9494 - val_accuracy:

Epoch 115/400
782/782 [==============================] - 3s 4ms/step - loss: 0.9062 - accuracy: 0.6831 - val_loss: 0.7687 - val_accuracy: 0.7341
Epoch 116/400
782/782 [==============================] - 3s 4ms/step - loss: 0.9085 - accuracy: 0.6825 - val_loss: 0.7606 - val_accuracy: 0.7384
Epoch 117/400
782/782 [==============================] - 3s 4ms/step - loss: 0.9096 - accuracy: 0.6803 - val_loss: 0.7780 - val_accuracy: 0.7316
Epoch 118/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8988 - accuracy: 0.6868 - val_loss: 0.7822 - val_accuracy: 0.7282
Epoch 119/400
782/782 [==============================] - 3s 4ms/step - loss: 0.9035 - accuracy: 0.6825 - val_loss: 0.7700 - val_accuracy: 0.7342
Epoch 120/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8957 - accuracy: 0.6852 - val_loss: 0.7575 - val_accuracy: 0.7407
Epoch 121/400
782/782 [==============================] - 3s 4ms/step - loss: 0.8965 - accuracy: 0.6852 - val_loss: 0.7482 - val_ac

782/782 [==============================] - 3s 4ms/step - loss: 0.7433 - accuracy: 0.7424 - val_loss: 0.6406 - val_accuracy: 0.7767
Epoch 228/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7414 - accuracy: 0.7431 - val_loss: 0.6596 - val_accuracy: 0.7677
Epoch 229/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7461 - accuracy: 0.7397 - val_loss: 0.6313 - val_accuracy: 0.7798
Epoch 230/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7450 - accuracy: 0.7420 - val_loss: 0.6432 - val_accuracy: 0.7761
Epoch 231/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7473 - accuracy: 0.7407 - val_loss: 0.6330 - val_accuracy: 0.7811
Epoch 232/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7423 - accuracy: 0.7426 - val_loss: 0.6234 - val_accuracy: 0.7844
Epoch 233/400
782/782 [==============================] - 3s 4ms/step - loss: 0.7424 - accuracy: 0.7421 - val_loss: 0.6428 - val_accuracy: 0.7770

782/782 [==============================] - 3s 4ms/step - loss: 0.6766 - accuracy: 0.7646 - val_loss: 0.5856 - val_accuracy: 0.7996
Epoch 340/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6760 - accuracy: 0.7656 - val_loss: 0.5705 - val_accuracy: 0.8045
Epoch 341/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6796 - accuracy: 0.7649 - val_loss: 0.5883 - val_accuracy: 0.7971
Epoch 342/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6723 - accuracy: 0.7659 - val_loss: 0.5764 - val_accuracy: 0.8024
Epoch 343/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6737 - accuracy: 0.7668 - val_loss: 0.5888 - val_accuracy: 0.7965
Epoch 344/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6744 - accuracy: 0.7677 - val_loss: 0.5615 - val_accuracy: 0.8041
Epoch 345/400
782/782 [==============================] - 3s 4ms/step - loss: 0.6713 - accuracy: 0.7687 - val_loss: 0.5680 - val_accuracy: 0.8029

In [32]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI_3Conv-PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of reduced base quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmptj_jkdf_/assets


INFO:tensorflow:Assets written to: /tmp/tmptj_jkdf_/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 17:40:20.765482: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 17:40:20.765512: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 17:40:20.765665: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmptj_jkdf_
2023-05-17 17:40:20.770867: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 17:40:20.770886: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmptj_jkdf_
2023-05-17 17:40:20.788132: I tensorflow/

== Input details ==
name: serving_default_conv2d_49_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of reduced base quantized to int8 model is 80.44%


In [33]:
c_model_name = 'cifar10_base_3conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [43]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_3Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,validation_data=(x_test, y_test),
                     callbacks=[checkpoint_callback])

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_8 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_61 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_42 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_56 (QuantizeW  (None, 16, 16, 32)       1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_62 (QuantizeWr  (None, 16, 16, 64)     

782/782 [==============================] - 9s 12ms/step - loss: 1.2621 - accuracy: 0.5440 - val_loss: 1.1612 - val_accuracy: 0.6095
Epoch 36/400
782/782 [==============================] - 9s 12ms/step - loss: 1.2489 - accuracy: 0.5489 - val_loss: 1.1636 - val_accuracy: 0.5932
Epoch 37/400
782/782 [==============================] - 9s 12ms/step - loss: 1.2448 - accuracy: 0.5492 - val_loss: 1.1526 - val_accuracy: 0.5989
Epoch 38/400
782/782 [==============================] - 10s 13ms/step - loss: 1.2350 - accuracy: 0.5545 - val_loss: 1.1312 - val_accuracy: 0.6096
Epoch 39/400
782/782 [==============================] - 9s 12ms/step - loss: 1.2275 - accuracy: 0.5579 - val_loss: 1.1239 - val_accuracy: 0.6158
Epoch 40/400
782/782 [==============================] - 9s 12ms/step - loss: 1.2220 - accuracy: 0.5629 - val_loss: 1.0980 - val_accuracy: 0.6270
Epoch 41/400
782/782 [==============================] - 9s 12ms/step - loss: 1.2126 - accuracy: 0.5626 - val_loss: 1.1197 - val_accuracy: 0.61

782/782 [==============================] - 7s 9ms/step - loss: 0.9728 - accuracy: 0.6578 - val_loss: 0.8339 - val_accuracy: 0.7102
Epoch 92/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9697 - accuracy: 0.6575 - val_loss: 0.8652 - val_accuracy: 0.7005
Epoch 93/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9657 - accuracy: 0.6574 - val_loss: 0.8385 - val_accuracy: 0.7105
Epoch 94/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9607 - accuracy: 0.6626 - val_loss: 0.8579 - val_accuracy: 0.6984
Epoch 95/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9598 - accuracy: 0.6619 - val_loss: 0.8605 - val_accuracy: 0.7007
Epoch 96/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9537 - accuracy: 0.6649 - val_loss: 0.8396 - val_accuracy: 0.7102
Epoch 97/400
782/782 [==============================] - 7s 9ms/step - loss: 0.9542 - accuracy: 0.6633 - val_loss: 0.8135 - val_accuracy: 0.7185
Epoch

782/782 [==============================] - 13s 17ms/step - loss: 0.8442 - accuracy: 0.7054 - val_loss: 0.7236 - val_accuracy: 0.7471
Epoch 148/400
782/782 [==============================] - 14s 18ms/step - loss: 0.8410 - accuracy: 0.7061 - val_loss: 0.7207 - val_accuracy: 0.7479
Epoch 149/400
782/782 [==============================] - 13s 17ms/step - loss: 0.8377 - accuracy: 0.7082 - val_loss: 0.7016 - val_accuracy: 0.7551
Epoch 150/400
782/782 [==============================] - 13s 17ms/step - loss: 0.8393 - accuracy: 0.7040 - val_loss: 0.7349 - val_accuracy: 0.7409
Epoch 151/400
782/782 [==============================] - 13s 17ms/step - loss: 0.8362 - accuracy: 0.7058 - val_loss: 0.6947 - val_accuracy: 0.7586
Epoch 152/400
782/782 [==============================] - 13s 17ms/step - loss: 0.8326 - accuracy: 0.7090 - val_loss: 0.7466 - val_accuracy: 0.7360
Epoch 153/400
782/782 [==============================] - 13s 17ms/step - loss: 0.8368 - accuracy: 0.7065 - val_loss: 0.7144 - val_ac

Epoch 203/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7684 - accuracy: 0.7310 - val_loss: 0.6639 - val_accuracy: 0.7690
Epoch 204/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7673 - accuracy: 0.7334 - val_loss: 0.6429 - val_accuracy: 0.7778
Epoch 205/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7668 - accuracy: 0.7337 - val_loss: 0.6405 - val_accuracy: 0.7781
Epoch 206/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7627 - accuracy: 0.7339 - val_loss: 0.6603 - val_accuracy: 0.7710
Epoch 207/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7646 - accuracy: 0.7320 - val_loss: 0.6685 - val_accuracy: 0.7648
Epoch 208/400
782/782 [==============================] - 14s 18ms/step - loss: 0.7625 - accuracy: 0.7355 - val_loss: 0.6664 - val_accuracy: 0.7723
Epoch 209/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7579 - accuracy: 0.7358 - val_loss: 0

Epoch 259/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7135 - accuracy: 0.7510 - val_loss: 0.6455 - val_accuracy: 0.7764
Epoch 260/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7180 - accuracy: 0.7518 - val_loss: 0.6325 - val_accuracy: 0.7802
Epoch 261/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7161 - accuracy: 0.7534 - val_loss: 0.6103 - val_accuracy: 0.7893
Epoch 262/400
782/782 [==============================] - 14s 18ms/step - loss: 0.7147 - accuracy: 0.7514 - val_loss: 0.6145 - val_accuracy: 0.7855
Epoch 263/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7130 - accuracy: 0.7533 - val_loss: 0.6250 - val_accuracy: 0.7825
Epoch 264/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7095 - accuracy: 0.7544 - val_loss: 0.6270 - val_accuracy: 0.7800
Epoch 265/400
782/782 [==============================] - 13s 17ms/step - loss: 0.7095 - accuracy: 0.7515 - val_loss: 0

Epoch 315/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6773 - accuracy: 0.7656 - val_loss: 0.5804 - val_accuracy: 0.7980
Epoch 316/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6827 - accuracy: 0.7620 - val_loss: 0.6237 - val_accuracy: 0.7821
Epoch 317/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6807 - accuracy: 0.7623 - val_loss: 0.6052 - val_accuracy: 0.7876
Epoch 318/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6800 - accuracy: 0.7645 - val_loss: 0.6034 - val_accuracy: 0.7892
Epoch 319/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6753 - accuracy: 0.7663 - val_loss: 0.5787 - val_accuracy: 0.7968
Epoch 320/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6762 - accuracy: 0.7645 - val_loss: 0.5803 - val_accuracy: 0.7974
Epoch 321/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6757 - accuracy: 0.7655 - val_loss: 0

Epoch 371/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6549 - accuracy: 0.7732 - val_loss: 0.5719 - val_accuracy: 0.7989
Epoch 372/400
782/782 [==============================] - 14s 17ms/step - loss: 0.6578 - accuracy: 0.7714 - val_loss: 0.5695 - val_accuracy: 0.7987
Epoch 373/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6505 - accuracy: 0.7735 - val_loss: 0.5628 - val_accuracy: 0.8039
Epoch 374/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6495 - accuracy: 0.7763 - val_loss: 0.5726 - val_accuracy: 0.7972
Epoch 375/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6524 - accuracy: 0.7739 - val_loss: 0.5557 - val_accuracy: 0.8051
Epoch 376/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6535 - accuracy: 0.7750 - val_loss: 0.5891 - val_accuracy: 0.7903
Epoch 377/400
782/782 [==============================] - 13s 17ms/step - loss: 0.6547 - accuracy: 0.7722 - val_loss: 0

In [46]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_3Conv_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_3Conv_qat_int8.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Quantization aware training loss:  0.577202320098877
Quantization aware training accuracy:  0.7972999811172485


INFO:tensorflow:Assets written to: /tmp/tmpi9v_dq7q/assets


INFO:tensorflow:Assets written to: /tmp/tmpi9v_dq7q/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 23:09:30.438235: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 23:09:30.438265: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 23:09:30.438414: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpi9v_dq7q
2023-05-17 23:09:30.445365: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 23:09:30.445386: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpi9v_dq7q
2023-05-17 23:09:30.464158: I tensorflow/

== Input details ==
name: serving_default_conv2d_61_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of quantized to int8 model is 79.67999999999999%


In [47]:
c_model_name = 'cifar10_base_3conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))